In [ ]:
pip install pandas

In [ ]:
pip install numpy

In [ ]:
pip install sklearn

In [ ]:
pip install matplotlib

In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv("Mumbai1.csv")
data = data.drop(["Unnamed: 0"],axis =1)
data.head()

In [ ]:
data = data.drop(["New/Resale","Intercom","Gas Connection","Jogging Track","Landscaped Gardens","Indoor Games","Maintenance Staff","24x7 Security","Children's Play Area"],axis =1)
data.head()

In [ ]:
data.isna().sum()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
data.Price.describe()

In [ ]:
plt.plot(data.Price);

In [ ]:
data.shape

In [ ]:
upper = data.Price.mean() + 3*data.Price.std()
lower = data.Price.mean() - 3*data.Price.std()

In [ ]:
data[(data.Price<lower) | (data.Price>upper)]

In [ ]:
data2= data[(data.Price>lower) & (data.Price<upper)]
data2.shape

In [ ]:
data.shape[0] - data2.shape[0]

In [ ]:
plt.plot(data2.Price);

In [ ]:
data2.head()

In [ ]:
data2.Area.describe()

In [ ]:
lower_area, upper_area = data2.Area.quantile([0.004,0.996])
upper_area

In [ ]:
data2.Area.describe()

In [ ]:
data2[(data2.Area<lower_area) | (data2.Area>upper_area)]

In [ ]:
data3 = data2[(data2.Area>lower_area) & (data2.Area<upper_area)]
data3.shape

In [ ]:
data2.shape[0] - data3.shape[0]

In [ ]:
data3.Area.describe()

In [ ]:
plt.plot(data3.Area);

In [ ]:
data3.shape

In [ ]:
data3.head(2)

In [ ]:
data3["No. of Bedrooms"].unique()

In [ ]:
upper_bhk = data3["No. of Bedrooms"].quantile(0.999)
data4 = data3[data3["No. of Bedrooms"]<upper_bhk]
data4.shape

In [ ]:
upper_bhk

In [ ]:
data4["No. of Bedrooms"].unique()

In [ ]:
data4.shape

In [ ]:
data4.head(2)

In [ ]:
data4.Location.nunique()

In [ ]:
loc_stats = data4.Location.value_counts(ascending=False)
loc_stats

In [ ]:
len(loc_stats[loc_stats==1])

In [ ]:
loc_stats_equal_to_1 = loc_stats[loc_stats==1]
len(loc_stats_equal_to_1)

In [ ]:
data4.Location = data4.Location.apply(lambda x : "other" if x in loc_stats_equal_to_1 else x)

In [ ]:
data4.shape

In [ ]:
data4.Location.nunique()

In [ ]:
data4.head(2)

In [ ]:
data4["Price_By_Area"] = data4.Price/data4.Area
data4.head(2)

In [ ]:
data4.Price_By_Area.describe()

In [ ]:
lo_pa, up_pa = data4.Price_By_Area.quantile([0.018,0.982])
up_pa

In [ ]:
import numpy as np
from scipy.stats import norm
plt.hist(data4.Price_By_Area, bins = 40, rwidth=0.5, density = True);
rng = np.arange(-10000,data4.Price_By_Area.max())
plt.plot(rng, norm.pdf(rng, data4.Price_By_Area.mean(), data4.Price_By_Area.std()));

In [ ]:
data5 = data4[(data4.Price_By_Area>lo_pa) & (data4.Price_By_Area<up_pa)]
data5.shape

In [ ]:
data4.shape[0] - data5.shape[0]

In [ ]:
plt.hist(data5.Price_By_Area, bins = 40, rwidth=0.5, density = True);
rng = np.arange(-10000,data5.Price_By_Area.max())
plt.plot(rng, norm.pdf(rng, data5.Price_By_Area.mean(), data5.Price_By_Area.std()));

In [ ]:
data5.shape

In [ ]:
data5.head()

In [ ]:
data5 = data5.drop(["Price_By_Area"],axis =1)
data5.head(3)

In [ ]:
data5.shape

In [ ]:
dummies = pd.get_dummies(data5.Location)
dummies.head(2)

In [ ]:
dummies = dummies.drop(["other"],axis =1)
dummies.head(2)

In [ ]:
data6 = pd.concat([data5,dummies],axis = 1)
data6.head(2)

In [ ]:
data7 = data6.drop(["Location"],axis =1)
data7.head()

In [ ]:
data7.shape

In [ ]:
X = data7.drop(["Price"],axis =1)
X.head()

In [ ]:
y = data7.Price
y.head()

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

In [ ]:
len(X_train)

In [ ]:
len(X_test)

In [ ]:
from sklearn.linear_model import Lasso

In [ ]:
model = Lasso()

In [ ]:
model.fit(X_train,y_train)

In [ ]:
model.score(X_test,y_test)

In [ ]:
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score

cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
score = cross_val_score(Lasso(), X,y, cv=cv)
score.mean()

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
def find_best_model_using_gridsearchcv(X,y):
    algos = {
        'linear_regression' : {
            'model': LinearRegression(),
            'params': {
                'normalize': [True, False]
            }
        },
        'lasso': {
            'model': Lasso(),
            'params': {
                'alpha': [1,2],
                'selection': ['random', 'cyclic']
            }
        },
        'decision_tree': {
            'model': DecisionTreeRegressor(),
            'params': {
                'criterion' : ['mse','friedman_mse'],
                'splitter': ['best','random']
            }
        }
    }
    scores = []
    cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
    for algo_name, config in algos.items():
        gs =  GridSearchCV(config['model'], config['params'], cv=cv, return_train_score=False)
        gs.fit(X,y)
        scores.append({
            'model': algo_name,
            'best_score': gs.best_score_,
            'best_params': gs.best_params_
        })

    return pd.DataFrame(scores,columns=['model','best_score','best_params'])

find_best_model_using_gridsearchcv(X,y)

In [ ]:
model = Lasso(alpha = 2, selection="cyclic")
model.fit(X_train,y_train)
model.score(X_test,y_test)

In [ ]:
pip install pickle

In [ ]:
import pickle

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
plt.scatter(y_test,y_pred);

In [ ]:
with open("mumbai_house_prices_prediction.pickle","wb") as f:
    pickle.dump(model,f)

In [ ]:
pip install json

In [ ]:
import json
columns ={
    'data_columns' : [col.lower() for col in X.columns]
}
with open('columns.json','w') as f:
    f.write(json.dumps(columns))